Advance Commands

# DATA READING

### Data Reading JSON

### Collect_List

In [ ]:
data = [('user1','book1'),
        ('user1','book2'),
        ('user2','book2'),
        ('user2','book4'),
        ('user3','book1')]

schema = 'user string, book string'

df_book = spark.createDataFrame(data,schema)

df_book.display()

In [ ]:
df_book.groupBy('user').agg(collect_list('book')).display()

In [ ]:
df.select('Item_Type','Outlet_Size','Item_MRP').display()

### PIVOT

In [ ]:
df.groupBy('Item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).display()

### When-Otherwise

#### Scenario - 1

In [ ]:
df = df.withColumn('veg_flag',when(col('Item_Type')=='Meat','Non-Veg').otherwise('Veg'))

In [ ]:
df.display()

In [ ]:
df.withColumn('veg_exp_flag',when(((col('veg_flag')=='Veg') & (col('Item_MRP')<100)),'Veg_Inexpensive')\
                            .when((col('veg_flag')=='Veg') & (col('Item_MRP')>100),'Veg_Expensive')\
                            .otherwise('Non_Veg')).display()

### JOINS

In [ ]:
dataj1 = [('1','gaur','d01'),
          ('2','kit','d02'),
          ('3','sam','d03'),
          ('4','tim','d03'),
          ('5','aman','d05'),
          ('6','nad','d06')]

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING'

df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR'),
          ('d02','Marketing'),
          ('d03','Accounts'),
          ('d04','IT'),
          ('d05','Finance')]

schemaj2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [ ]:
df1.display()

In [ ]:
df2.display()

#### Inner Join

In [ ]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').display()

#### Left Join

In [ ]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'left').display()

#### LEFT JOIN

In [ ]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'right').display()

#### ANTI JOIN

In [ ]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'anti').display()

### WINDOW FUNCTIONS

#### ROW_NUMBER()

In [ ]:
df.display()

In [ ]:
from pyspark.sql.window import Window

In [ ]:
df.withColumn('rowCol',row_number().over(Window.orderBy('Item_Identifier'))).display()

#### RANK VS DENSE RANK

In [ ]:
df.withColumn('rank',rank().over(Window.orderBy(col('Item_Identifier').desc())))\
        .withColumn('denseRank',dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).display()

In [ ]:
df.withColumn('dum',sum('Item_MRP').over(Window.orderBy('Item_Identifier').rowsBetween(Window.unboundedPreceding,Window.currentRow))).display()

#### Cumulative Sum

In [ ]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type'))).display()

In [ ]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.currentRow))).display()

In [ ]:
df.withColumn('totalsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).display()

### USER DEFINED FUNCTIONS (UDF)

#### STEP - 1

In [ ]:
def my_func(x):
    return x*x

#### STEP - 2

In [ ]:
my_udf = udf(my_func)

In [ ]:
df.withColumn('mynewcol',my_udf('Item_MRP')).display()

### DATA WRITING

#### CSV

In [ ]:
df.write.format('csv')\
        .save('/FileStore/tables/CSV/data.csv')

#### APPEND

In [ ]:
df.write.format('csv')\
        .mode('append')\
        .save('/FileStore/tables/CSV/data.csv')

In [ ]:
df.write.format('csv')\
        .mode('append')\
        .option('path','/FileStore/tables/CSV/data.csv')\
        .save()

#### Overwrite

In [ ]:
df.write.format('csv')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### Error

In [ ]:
df.write.format('csv')\
.mode('error')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### Ignore

In [ ]:
df.write.format('csv')\
.mode('ignore')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### PARQUET

In [ ]:
df.write.format('parquet')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### TABLE

In [ ]:
df.write.format('parquet')\
.mode('overwrite')\
.saveAsTable('my_table')

In [ ]:
df.display()

### SPARK SQL

#### createTempView

In [ ]:
df.createTempView('my_view')

In [ ]:
%sql

select * from my_view where Item_Fat_Content = 'Lf'

In [ ]:
df_sql = spark.sql("select * from my_view where Item_Fat_Content = 'Lf'")

In [ ]:
df_sql.display()